In [1]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [2]:
# Display the shapes of the data.
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (50000, 32, 32, 3)
Shape of X_test: (10000, 32, 32, 3)
Shape of y_train: (50000, 1)
Shape of y_test: (10000, 1)


In [3]:
from layers_activation import ReluLayer
from layers_convolution import ConvolutionLayer
from layers_pooling import PoolLayer
from layers_flatten import Flatten
from layers_dense import DenseLayer
from loss import SoftmaxLayer
from network import Model

In [4]:
model = Model(
    ConvolutionLayer(filters=3, filter_size=3, padding=1, stride=1),
    ReluLayer(),
    PoolLayer(filter_size=2, stride=2, mode='max'),
    Flatten(),
    DenseLayer(units=100),
    SoftmaxLayer()
)

In [5]:
y_train.shape

(50000, 1)

In [6]:
model.train(X_train, y_train, 0.003, batch_size=64, epochs=100)

Running epoch: 1
mini_batch[0] shape (64, 32, 32, 3)
mini_batch[1] shape (64, 1)
mini_batch_preds final shape (100,)
mini_batch_preds: [0.00956877 0.00902774 0.00965791 0.01061477 0.0109384  0.00921932
 0.01013806 0.0107698  0.00944624 0.00921667 0.00990776 0.01006448
 0.00950734 0.01047794 0.00994679 0.00912854 0.00914698 0.00906448
 0.0102353  0.01086245 0.00908557 0.00936945 0.01073225 0.00917537
 0.01005484 0.00993618 0.01072059 0.00900335 0.0101031  0.01027485
 0.01067288 0.01057358 0.00915459 0.01046742 0.01053856 0.01013742
 0.01039644 0.01003487 0.01081703 0.00923794 0.01026403 0.00937052
 0.01001428 0.01090349 0.01077021 0.01001317 0.00962187 0.01067936
 0.01053325 0.00933215 0.01013089 0.00982409 0.01017995 0.00916662
 0.01084997 0.01048378 0.0107894  0.01075777 0.00972165 0.00922338
 0.01055545 0.01022547 0.00926055 0.0089656  0.01094002 0.00955923
 0.010206   0.01053022 0.00921784 0.00991733 0.01037962 0.01088602
 0.01018144 0.0101215  0.00920011 0.00938341 0.00921568 0.009

ValueError: shapes (64,100) and (49152,) not aligned: 100 (dim 1) != 49152 (dim 0)

In [12]:
import tensorflow as tf

# one hot encode target values
trainY = tf.keras.utils.to_categorical(y_train)
testY = tf.keras.utils.to_categorical(y_test)

In [15]:
trainY[:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [17]:
import numpy as np
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [53]:

def to_categorical(labels, num_classes):
    """
    Converts image labels to one-hot encoded binary vectors with length of num_classes.
    Arguments:
        labels -- image classification labels represented as index in a list, integer.
        num_classes -- number of classes in image classification task, integer.
    Returns:
        label_vec -- binary vector representation of the class, with 1 for correct label, 0's otherwise. 
    """
    # Initialize a zero matrix with the correct dimensions.
    label_vec = np.zeros((len(labels), num_classes))
    # Loop through the labels in the input.
    for i, label in enumerate(labels):
        # Replace the 0 at the given location with a 1 for the correct class.
        # Subtract 1 to account for indexing
        label_vec[0][label - 1] = 1
    return label_vec

In [58]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [54]:
my_trainY = to_categorical(y_train, num_classes=10)

IndexError: index 255 is out of bounds for axis 0 with size 10

In [33]:
my_trainY[:5]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [61]:
label_vec = np.zeros((len(y_train), 10))
label_vec

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [62]:
len(label_vec)

50000

In [63]:
label_vec[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [39]:
tensor[3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])